<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/week3/7%ED%8C%80/%EB%B2%84%EC%8A%A4_%EC%9A%B4%ED%96%89%EC%8B%9C%EA%B0%84_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 버스 운행시간 데이터를 활용한 데이터 분석
- 버스가 다음 정류장에 도착하기 위해서 걸릴 시간을 예측

## 1. 라이브러리 및 데이터
## Library & Data

In [4]:
## matplotlib 사용시 한국어 나오게 하는 코드(큰 의미 없음)

!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)

# fm._rebuild() 이거 있으니 실행이 안되서 빼고 진행하겠음.

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic') 

In [5]:
import pandas as pd #판다스 패키지 불러오기
import lightgbm as lgb
import folium

In [6]:
# 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/229611_버스 운행시간 예측 경진대회_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/229611_버스 운행시간 예측 경진대회_data/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/229611_버스 운행시간 예측 경진대회_data/submission.csv')

## 2. 탐색적 자료분석
## Exploratory Data Analysis (EDA)


### pd.DataFrame.head()
 - 데이터 프레임의 위에서 부터 n개 행을 보여주는 함수
 - n의 기본 값(default 값)은 5

In [7]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


+ id : 고유 id
+ date : 버스 운행 날짜
+ route_id : 버스 노선 ID
+ vh_id : 버스 id
+ route_nm : 버스 노선 실제 번호
+ now_latitude : 현재 정류소의 위도
+ now_longitude : 현재 정류소의 경도
+ now_station : 현재 정류소 이름
+ now_arrive_time : 현재 정류장에 도착한 시간
+ distance : 현재 정류장에서 다음 정류장까지 실제 이동한 거리
+ next_station : 다음 정류소 이름
+ next_latitude : 다음 정류소의 위도
+ next_longitude : 다음 정류소의 경도
+ next_arrive_time : 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)으로 답안 제출을 위해서 예측해야 되는 값



In [8]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [9]:
submission.head()

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0


### pd.DataFrame.tail()
 - 데이터 프레임의 아래에서 부터 n개 행을 보여주는 함수
 - n의 기본 값(default 값)은 5

In [10]:
train.tail()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38
210456,210456,2019-10-28,405328102,7983486,281-2,33.248487,126.511195,대륜동주민센터,21시,189.0,서귀포버스터미널(가상정류소),33.249091,126.509224,24


In [11]:
print(train['date'].min())
print(train['date'].max())
print(test['date'].min())
print(test['date'].max())

2019-10-15
2019-10-28
2019-10-29
2019-11-05


### pd.DataFrame.shape
 - 데이터 프레임의 행의 개수와 열의 개수가 저장되어 있는 속성(attribute)

In [12]:
print(train.shape)
print(test.shape)
print(submission.shape)

(210457, 14)
(91774, 13)
(91774, 2)


### pd.DataFrame.info()
- 데이터셋의 column별 정보를 알려주는 함수
- 비어 있지 않은 값은 (non-null)은 몇개인지?
- column의 type은 무엇인지?
 - type의 종류 : int(정수), float(실수), object(문자열), 등등 (date, ...)

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


### pd.DataFrame.describe()
- 숫자형 (int, float) column들의 기술 통계량을 보여주는 함수

- 기술통계량이란?
 - 해당 column을 대표할 수 있는 통계값들을 의미
 
 
- 기술통계량 종류
 - count: 해당 column에서 비어 있지 않은 값의 개수
 - mean: 평균
 - std: 표준편차
 - min: 최솟값 (이상치 포함)
 - 25% (Q1): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 1/4번째 지점에 있는 값
 - 50% (Q2): 중앙값 (전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 2/4번째 지점에 있는 값)
 - 75% (Q3): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 3/4번째 지점에 있는 값
 - max: 최댓값 (이상치 포함) 
 
 
 
- 이상치: 울타리 밖에 있는 부분을 이상치라고 정의함
   - 아래쪽 울타리: $Q_1$ - $1.5 * IQR$
   - 위쪽 울타리: $Q_3$ + $1.5 * IQR$
   - $IQR$ = $Q_3 - Q_1$
 
 
<img src="https://miro.medium.com/max/10125/1*NRlqiZGQdsIyAu0KzP7LaQ.png" width="700" height="500">

In [15]:
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


### pd.DataFrame.groupby()
 - 집단에 대한 통계량 확인 
 
<img src="https://s3.amazonaws.com/files.dezyre.com/images/Tutorials/Split+Apply+Combine+Strategy.png" width="700" height="500">

In [16]:
len(train['now_station'].unique()) # 정류장 개수

348

In [17]:
train[['route_id', 'vh_id']].drop_duplicates()   #중복 제거

,route_id,vh_id
0,405136001,7997025
181,405136001,7997034
381,405136001,7997035
539,405136001,7997040
717,405136001,7997044
...,...,...
20226,405136522,7997004
26037,405320116,7983225
27098,405320122,7983067
45506,405328102,7983401


In [18]:
train[['route_id', 'vh_id']].drop_duplicates().groupby('route_id').count()   #중복 제거후 통계량 구해줌.

,vh_id
route_id,
405136001,10
405136002,5
405136007,1
405136012,3
405136521,6
405136522,15
405320111,3
405320112,5
405320113,1


In [19]:
train[['route_id', 'next_arrive_time']].drop_duplicates().groupby('route_id').mean()

,next_arrive_time
route_id,
405136001,214.493827
405136002,494.006873
405136007,151.703125
405136012,521.729231
405136521,596.943262
405136522,439.278846
405320111,206.722543
405320112,228.140704
405320113,163.738462


### folium.Map()
 - location 옵션에 있는 좌표를 중심으로 지도 시각화
 - 도화지 위에 레이어 겹겹이 쌓는 형식

In [20]:
coord = [35.1559361, 129.1384361]  #시각화할 좌표 (부산) 
map_osm = folium.Map(location = coord)
map_osm

In [21]:
map_osm = folium.Map(location=coord, zoom_start=15) # 시작하자마자 확대. default값이 10이니 그보다 크기만 하면 됨.
map_osm

In [22]:
#함수에 커서 올리고 ctrl 스페이스바 누르면 설명 볼 수 있음. 
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain') # 배경 변경하는 하나의 옵션

map_osm

In [23]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Toner')
map_osm

In [24]:
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location=coord, zoom_start=12)

folium.Marker(coord, popup='부산대학교', tooltip = '부산대학교 tooltip').add_to(map_osm) # 마커 생성 뒤 지도에 추가해주는 기능
# 마커 좌표, 팝업(안내문구)설정, 마우스 호버링(클릭은 안하고 갖다 대기만 했을때)시 표시하는 툴팁. add_to로 지도레이어 위에다가 찍기.

# folium이 한국어 호환이 잘 안되서 한글 깨짌 있는데, html로 한번 저장하면 됨.
map_osm.save('index.html')

map_osm

마커 아이콘 종류 찾아볼 수 있는 곳.      
default images: https://getbootstrap.com/docs/3.3/components/

- 검색 기능이 없다

fontawesome:https://fontawesome.com/icons?d=gallery

- 해당링크의 무료인것만 쓸 수 있음

In [25]:
map_osm = folium.Map(location=coord, zoom_start=12)
folium.Marker(coord, popup='부산대학교', icon=folium.Icon(icon='star')).add_to(map_osm)
# 마커의 아이콘도 바꿀 수 있음. 
map_osm

In [26]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.Marker([37.566345, 126.977893], popup='서울특별시청', icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
# 마커의 아이콘, 색상 변환
folium.Circle([37.4600, 126.9519], tooltip='서울대학교', radius=3000, color="green").add_to(map_osm)
# 지역에 동그라미 표시하는 법
folium.CircleMarker([37.5662, 126.9386], radius=10, popup='연세대학교',color="black", fill_color="crimson").add_to(map_osm)
# 원 형태의 마커.
map_osm

In [27]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.RegularPolygonMarker([37.566345, 126.977893], popup='서울특별시청', fill_color='red',color='red', number_of_sides=3, radius=30, fill_opacity=0.2).add_to(map_osm)
# 다각형 마커를 그리는 방법(삼각형), number_of_side = n각형 fill_opacity = 투명도
folium.RegularPolygonMarker([37.4600, 126.9519], popup='서울대학교', fill_color='green',color='green', number_of_sides=4, radius=10, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.5662, 126.9386], popup='연세대학교',fill_color='blue',color='blue', number_of_sides=6, radius=10).add_to(map_osm)
map_osm

In [28]:
# 해당 노선에 있는 모든 정류장들을 마커로 찍는 코드
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route = map_data[map_data['route_id'] == 405320122].drop_duplicates("now_station")
# route_id를 수정하며, nextarrivetime이 긴곳은 번화가를 다니는 노선, 짧은곳은 외곽을 다니는 노선 확인.


map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)


for item in map_bus_route.index:
    lat = map_bus_route.loc[item,'now_latitude']
    long = map_bus_route.loc[item,'now_longitude']
    
    folium.Marker([lat,long], popup = map_bus_route.loc[item,'now_station'],
                      icon = folium.Icon(color = 'red', icon = 'info-sign')
                      ).add_to(map_osm)

map_osm.save('index.html')

map_osm

## 3. 데이터 전처리
## Data Cleansing & Pre-Processing  
## 이 데이터에선 결측값이 없으므로, 데이터 슬라이싱만 함.

In [48]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,6,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,6,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,6,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),6,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,7,374.0,은남동,33.485822,126.490897,64


### pd.Series.str.slice()
- Series 내에 문자열을 일괄적으로 slicing (자르기)

In [46]:
train['now_arrive_time'] = train['now_arrive_time'].str.slice(stop=-1).astype('int') #마지막 문자 전까지만 자르기/ astype를 활용해서 object가 아닌 int 형으로 바꾸기.
# 한번 실행한 후에 다시 실행시키면 실행 안됨(str대상 작업인데 이미 int형 객체로 바뀌었기 때문.)

AttributeError: ignored

In [49]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [50]:
test['now_arrive_time'] = test['now_arrive_time'].str.slice(stop=-1).astype('int')

In [51]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,7,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,7,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,8,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,8,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,8,550.0,케이티앤지,33.482077,126.485355


## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [53]:
features = ['now_latitude', 'now_longitude', 'now_arrive_time', 'distance']
target = ['next_arrive_time']

In [64]:
X_train, y_train, X_test = train[features], train[target], test[features]

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [65]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [66]:
model_dict = {
    'linear':LinearRegression(), 
    'rf':RandomForestRegressor(random_state=0, n_jobs=-1),
    'lgbm':lgb.LGBMRegressor(random_state=0),
}

In [67]:
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [68]:
model_result = {}

for key in model_dict.keys():
  print('훈련중')
  model_dict[key].fit(X_train, y_train)
  print('예측중')
  model_result[key] = model_dict[key].predict(X_test)

훈련중
예측중
훈련중


<ipython-input-68-133a680eedbc>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(X_train, y_train)


예측중
훈련중
예측중


In [69]:
model_result

{'linear': array([[ 80.75162733],
        [ 86.59637449],
        [ 96.03730695],
        ...,
        [ 93.13399549],
        [ 72.93764136],
        [103.28646517]]),
 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
         31.84467172,  88.46184286]),
 'lgbm': array([ 42.12400643,  60.24878305, 120.81241189, ..., 105.81387439,
         30.82952485, 111.66787953])}

In [70]:
lr_submit = submission.copy()
rf_submit = submission.copy()
lgbm_submit = submission.copy()

In [74]:
lr_submit['next_arrive_time'] = model_result['linear']
rf_submit['next_arrive_time'] = model_result['rf']
lgbm_submit['next_arrive_time'] = model_result['lgbm']

In [75]:
lr_submit.to_csv('Ir_submit.csv', index=False)
rf_submit.to_csv('rf_submit.csv', index=False)
lgbm_submit.to_csv('lgbm_submit.csv', index=False)

## 6. 결과 및 결언
## Conclusion & Discussion